<a href="https://colab.research.google.com/github/pmontman/tmp_choicemodels/blob/main/nb/tutorials/WK_06_tuto_ordered_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 6: Ordered Logit

We will see an example on how to use biogeme to estimate ordered logit models.

We will use the (in)famous 'affairs' dataset, described below.

The data comes from a survey from *Psychology Today*, conducted in the year 1969. It describes 'extramarital affairs', specifically, the participants in 
the survey were asked this question:

How often engaged in extramarital sexual intercourse during the past year?

With the possible answers:

* None
* Once
* Twice
* Three times
* 4 to 10 times
* Monthly
* Weekly
* Daily

Then by measuring socioeconomic variables of the participants, we could try to model the utility that they receive from having an extramarital affair (or more than one).

We can see that this falls perfectly into the ordered logit, the alternatives have a clear notion of order, from less affairs per year to more affairs per year, yet they are not perfectly numerical (which would allow us to use standard regression techniques).


# Description of the dataset

A scientific paper making use of the dataset to develop a 'Theory of Extramarial Affairs' can be found [here](https://www.uibk.ac.at/econometrics/data/fair78.pdf).

The dataset has 601 observations of the following variables:

* **affairs:** The answer to the survey, the answers are encoded with numbers.
  0 = None, 1 = Once, 2 = Twice, 3=Three times, 7= 4 to 10 times, 12= monthly, 12 = weekly, 12 = daily.
  As we see, the information was encoded in such a way that we lose information about the more frequent answers, and the encoded numbers do not completely coincide with the frequencies. However, there is an ordinal relationship among the possible answer, from less frequent to more frequent. Several categories overlap in how the are described (from monthly to daily).

*  **gender:** Categorical variable indicating either male or female among the participants.

* **age:** Numeric variable coding age in years: 17.5 = under 20, 22 = 20–24, 27 = 25–29, 32 = 30–34, 37 = 35–39, 42 = 40–44, 47 = 45–49, 52 = 50–54, 57 = 55 or over. 

* **yearsmarried:** Numeric variable coding number of years married: 0.125 = 3 months or less, 0.417 = 4–6 months, 0.75 = 6 months–1 year, 1.5 = 1–2 years, 4 = 3–5 years, 7 = 6–8 years, 10 = 9–11 years, 15 = 12 or more years.

* **children:** Categorical variable indicating if there are children in the marriage.

* **religiousness:** Categorical variable indicating how religious in the person, encoded as numbers: 1 = anti, 2 = not at all, 3 = slightly, 4 = somewhat, 5 = very.

* **education:**: Categorical variable indicating the level of education. Encoded as numbers: 9 = grade school, 12 = high school graduate, 14 = some college, 16 = college graduate, 17 = some graduate work, 18 = master's degree, 20 = Ph.D., M.D., or other advanced degree.

* **occupation:** Categorical variable classifying the profession of the individual. Encoded as numbers, and the meaning of the numbers has been somewhat lost in time. But it could be something like the one in this [link.](https://dictionary.fitbir.nih.gov/portal/publicData/dataElementAction!view.action?dataElementName=HollingsheadJobClassCat&publicArea=true)

* **rating:** Categorical variable indicating how happy they are with the marriage. Encoded as numbers: 1 = very unhappy, 2 = somewhat unhappy, 3 = average, 4 = happier than average, 5 = very happy.

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [1]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.1 MB/s 
  Created wheel for biogeme: filename=biogeme-3.2.10-cp37-cp37m-linux_x86_64.whl size=4253607 sha256=14a10519ff3e119816b4dc6c327de22ff6f15700c03ca99bae1249b56189e0f5
  Stored in directory: /root/.cache/pip/wheels/5b/92/9b/63caa7ad9b2cd582de77d3701d10f7e8d041466f4a9d07d554
Successfully built biogeme


Load the packages, feel free to change the names.

In [2]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [3]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/affairs.csv'
affairs_pd = pd.read_csv(path)

A simple look at the dataset.

In [4]:
affairs_pd.head(5)

,affairs,gender,age,yearsmarried,children,religiousness,education,occupation,rating
0,0,male,37.0,10.00,no,3,18,7,4
1,0,female,27.0,4.00,no,4,14,6,4
2,0,female,32.0,15.00,yes,1,12,1,4
3,0,male,57.0,15.00,yes,5,18,6,5
4,0,male,22.0,0.75,no,2,17,6,3


# Auxiliary function

In [5]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Biogeme does not accept non-numerical variables.


We need to transform categorical variables that are not encoded into numbers.
The pandas function `factorize()` encodes a variable into integers. In this case, we will get 0 for Male, 1 for Female for the gender variable. We will get 0 for no, 1 for yes in the children varible. This means that we can use the encoding already as dummy variables that can be interpreted as 'isFemale' and 'hasChildren'.

In [6]:
affairs_pd['gender'] = affairs_pd['gender'].factorize()[0]
affairs_pd['children'] = affairs_pd['children'].factorize()[0]


After this transformation, we can already pass the pandas dataframe to biogeme, though be mindful that some variables such as occupation are numeric but should be transformed to dummy. 

# Ordinal logit with biogeme

We will first estimate the ordered logit 'manually' using biogeme, to get a better view of the process and how it is connected to the theory. Later we can create some auxiliary functions to simplify the process.

We start by update the globals as usual.

In [7]:
qbus_update_globals_bgm(affairs_pd)

### Utility function

The first step is to create the utility function, an important difference with respect to what we have done in the multinomial logit(MNL) and nested logit is that there will be only one utility function, while in the MNL there is one utility function per alternative.

As for the variables, after some consideration (maybe you disagree), we can take
the variables religiousness, rating and education as numeric, even though they are categorical. This is because religiousness can be represented as 'intensity' of the belief in religion and education can be roughly interpreted as 'years spent in education'. So for the sake of simplicity

In [8]:
# Parameters to be estimated
B_gender = exp.Beta('B_gender', 0, None, None, 0)
B_age = exp.Beta('B_age', 0, None, None, 0)
B_yearsmarried = exp.Beta('B_yearsmarried', 0, None, None, 0)
B_children = exp.Beta('B_children', 0, None, None, 0)
B_religiousness = exp.Beta('B_religiousness', 0, None, None, 0)
B_education = exp.Beta('B_education', 0, None, None, 0)
B_rating = exp.Beta('B_rating', 0, None, None, 0)

Define the one utility function using our familiar syntax.
We will simplify and not apply any transformation or interaction, though it might very well improve the fitting.

In [9]:
V_one = B_gender*gender + B_age*age + B_yearsmarried*yearsmarried + B_children*children + B_religiousness*religiousness + B_education*education + B_rating*rating

### Cutoff points

An important part of the ordinal logit model are the cut-off points, the values that will split the utility into different alternatives. These are being estimated from the data. 
Remember that we have to impose some order in the cutoff points, this is, tau1
is smaller than tau2, tau2 is smaller than tau3 and so on.

The way to do this in biogeme (and other estimation software) is to create some 'auxiliary' parameters that can be interpreted as the deltas or differences between taus. These deltas are created with `exp.Beta` as the any other parameter of the model.
 We impose that the deltas can be estimated, but they have to be greater than 0. Remember that when we define the parameters in Biogeme, one of the arguments of the function `exp.Beta` are the bounds that restrict the range to which the values can be estimated.

We can 'recreate' the taus from the deltas, for example, tau2 = tau1 + delta2, tau3 = tau2 + delta3 (or alternatively, tau3 = tau1 + delta2 + delta3).

Just to clarify: what we are doing with the deltas is imposing the order in the taus, it is just a trick to impose: all taus are in increasing order, starting from tau1, 'tau1' can have any value, tau2 has to be between tau1 and tau3, tau3 has to be between tau2 and tau4, ..., tau6 has to be between tau5 and +infinity. What we are doing is just a way to pass the information to biogeme.

In the next cell we declare the taus and the deltas. Notice the third argument in the `exp.Beta` when declaring the deltas is set to 0, the third argument specifies the lower bound in the estimation range. Notice how the taus are defined as the 'taus before them' plus some delta. With the exception of tau1. 
The default value of tau1 is -1, but this is arbitrary.

There next cell will be very verbose, because we have to manually define each of the deltas and taus. This is a candidate to be done automatically in an auxiliary function...

In [10]:
tau1 = exp.Beta('tau1', -1, 0, None, 0)

delta2 = exp.Beta('delta2', 1, 0, None, 0)
tau2 = tau1 + delta2

delta3 = exp.Beta('delta3', 2, 0, None, 0)
tau3 = tau2 + delta3

delta4 = exp.Beta('delta4', 3, 0, None, 0)
tau4 = tau3 + delta4

delta5 = exp.Beta('delta5', 4, 0, None, 0)
tau5 = tau4 + delta5

What is left to define is the 'model' in the biogeme, the equivalent of the 
biogeme `models.loglogit` for the multinomial logit. In this case, we have to do it manually (another candidate for auxiliary functions).

We define the 'model' in biogeme using a dictionary. The dictionary maps the **values for the choice variable** to the computed choice probabilities. The choice variable here is affairs.

In the next cell the function `dist.logisticcdf` is just the name of the logistic transform in biogeme (there is the logistic probability distribution, which the cumulative distribution function 'logisticcdf' being the equivalent logistic transform).

In [11]:
alt_probs_map = {
    0: dist.logisticcdf(tau1 - V_one),
    1: dist.logisticcdf(tau2 - V_one) - dist.logisticcdf( tau1 - V_one),
    2: dist.logisticcdf(tau3 - V_one) - dist.logisticcdf( tau2 - V_one),
    3: dist.logisticcdf(tau4 - V_one) - dist.logisticcdf( tau3 - V_one),
    7: dist.logisticcdf(tau5 - V_one) - dist.logisticcdf( tau4 - V_one),
    12: 1- dist.logisticcdf(tau5 - V_one)}

Then we take the log of the choice probabilities (for the loglikelihood) and specify which variable in the dataset contains the choice or alternatives.

In [12]:

logprob = exp.log(exp.Elem(alt_probs_map, affairs))

We declare the biogeme object as usual, and we estimate the model.

In [13]:
affairs_db = db.Database('affairs', affairs_pd)

biogeme  = bio.BIOGEME(affairs_db, logprob)

results = biogeme.estimate()

[06:18:07] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]
[06:18:07] < Warning >   Cannot read file __biogemeModelDefaultName.iter. Statement is ignored.
[06:18:07] < Warning >   Initial point not feasible. It will be projected onto the feasible domain.


And here are the results, for your interpretation!
The new part are the taus and deltas. We have tau1, and then the deltas that will allow us to recover the remaining taus. We check tha everything looks OK,all deltas are positive.

In [14]:
results.getEstimatedParameters()

,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
B_age,-0.034204,0.0,0.017692,-1.933310,5.319796e-02
B_children,0.380194,0.0,0.317240,1.198444,2.307442e-01
B_education,0.095778,0.0,0.049239,1.945169,5.175461e-02
B_gender,-0.072075,0.0,0.224468,-0.321093,7.481401e-01
B_rating,-0.454255,0.0,0.090662,-5.010407,5.431494e-07
B_religiousness,-0.311157,0.0,0.090100,-3.453457,5.534506e-04
B_yearsmarried,0.101443,0.0,0.032843,3.088692,2.010395e-03
delta2,0.368621,0.0,0.059607,6.184153,6.243668e-10
delta3,0.217145,0.0,0.052117,4.166477,3.093435e-05
delta4,0.276374,0.0,0.062076,4.452168,8.500749e-06


Simulation is as usual, pass the biogeme database and the dictionary with the targets to compute. In this case, to get the choice probabilities, we can reuse the `alt_probs_map` dictionary as the targets. 


In [15]:
  bgm_pred_model = bio.BIOGEME(affairs_db, alt_probs_map)
  simulatedValues = bgm_pred_model.simulate(results.getBetaValues())
  simulatedValues

,0,1,2,3,7,12
0,0.782045,0.056338,0.027309,0.029017,0.059046,0.046246
1,0.909749,0.026039,0.011877,0.012129,0.023239,0.016967
2,0.560606,0.087846,0.047773,0.055113,0.128663,0.119999
3,0.895750,0.029747,0.013657,0.014004,0.026995,0.019846
4,0.737549,0.064934,0.032178,0.034709,0.072329,0.058300
...,...,...,...,...,...,...
596,0.839328,0.043745,0.020625,0.021515,0.042556,0.032231
597,0.619476,0.082337,0.043371,0.048858,0.109406,0.096552
598,0.724178,0.067307,0.033579,0.036389,0.076399,0.062147
599,0.445266,0.091867,0.053347,0.064799,0.166582,0.178140


To simulate the utilities we have to create a dictionary just with the utility funcion. 

In [16]:
V_pred_map = {
    'util': V_one }

We do the simulation as usual, notice how the more positive the utility, the more it tends towars more infidelities. This is because we considered arbitrarily to intepret the 'utility that is received from having an affair'. We could have also used the 'utility from being faithful'.

In [17]:
  bgm_pred_util = bio.BIOGEME(affairs_db, V_pred_map)
  simulated_util = bgm_pred_util.simulate(results.getBetaValues())
  simulated_util

[06:18:08] < Warning >   Parameter delta2 not present in the model
[06:18:08] < Warning >   Parameter delta3 not present in the model
[06:18:08] < Warning >   Parameter delta4 not present in the model
[06:18:08] < Warning >   Parameter delta5 not present in the model
[06:18:08] < Warning >   Parameter tau1 not present in the model


,util
0,-1.277622
1,-2.310579
2,-0.243622
3,-2.150868
4,-1.033269
...,...
596,-1.653236
597,-0.487326
598,-0.965284
599,0.219817


# Exercise 1) Create a more fine tuned ordered logit model



# Exercise 2) Compare the predictions and accuracy of the fine tuned model to a multinomial logit